In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps



# animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# My username and password and CRUD Python module name

user = "aacuser"
passwd = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(user, passwd)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Robert Lowrey')

#Adding in Grazioso Salvare’s logo
image_filename = 'ProjectLogo.png' # replace with my image file name
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src ='data:image/png;base64,{}'.format(encoded_image.decode())),#placing HTML image tag
    html.Center(html.B(html.H2('Robert Lowrey'))), #Including name as my unique identifier 
    html.Hr(),
    html.Div(
        
# Code for the interactive filtering options
    className = 'Row',
            style = {'display': 'flex'},
            children ={
                html.Button(id = 'button1', n_clicks = 0, children = 'Water Rescue'), #Water Rescue filtering button
                html.Button(id = 'button2', n_clicks = 0, children = 'Mountain or Wilderness Rescue'), #Mountain or Wilderness Rescue button
                html.Button(id = 'button3', n_clicks = 0, children = 'Disaster or Individual Tracking'), #Disaster or Individual Tracking button
                html.Button(id = 'button4', n_clicks = 0, children = 'Reset') #Reset button
            }

    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # Setting up the features for the interactive data table to make it user-friendly for the client
        #completed code from Module Six milestone
        page_size = 15,
        row_selectable = "single",
        
                         

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback([Output('datatable-id','data')], #setting callback for button option being pressed
              [Input('button1', 'n_clicks'), Input ('button2', 'n-clicks'), Input ('button3', 'n_clicks'), Input('button4', 'n_clicks')])
def update_dashboard(button1, button2, button3, button4):
    if (int(button1) >= 1): #if button1 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Labrador Retriever Mix'},{'breed' : 'Chesapeake Bay Retriever'},{'breed' : 'Newfoundland'}]}, #will search these breeds
            {'sex_upon_outcome': 'Intact Female'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Female with training age between 26-156 weeks
        button2, button3, button4 = 0 #setting the other button options to 0
        
    elif (int(button2) >= 1): #if button2 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'German Shepherd'}, {'breed' : 'Alaskan Malamute'}, {'breed' : 'Old English Sheepdog'}, {'breed' : 'Sierian Husky'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button3, button4 =0 #set other button options to 0
    
    elif (int(button3) >= 1): #if button3 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Doberman Pinscher'}, {'breed' : 'German Shepherd'}, {'breed' : 'Golden Retriever'}, {'breed' : 'Bloodhound'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 20, 'gte': 300}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button2, button4 = 0 #set other button options to 0
    
    elif (int(button4)>= 1): #if button4 is chosen, 
        df = pd.DataFrame.from_records(db.read()), #reset the read method
        button1, button2, button3 = 0 #set other button options to zero
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns] #can select columns but not delete them
    data=df.to_dict('records') #setting data to dictionary

    return (data,columns) #return data and columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [ #adding code for pie chart that responds to filtering options
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


ServerSelectionTimeoutError: nv-desktop-services.apporto.com:30182: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 6575183fdbcb715324af5892, topology_type: Single, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 30182) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:30182: [Errno 111] Connection refused')>]>

In [3]:
pip install pymongo jupyter_dash dash_leaflet dash plotly dash-table base64 numpy pandas matplotlib

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement base64 (from versions: none)
ERROR: No matching distribution found for base64
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps



# animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# My username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Robert Lowrey')

#Adding in Grazioso Salvare’s logo
image_filename = 'ProjectLogo.png' # replace with my image file name
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src ='data:image/png;base64,{}'.format(encoded_image.decode())),#placing HTML image tag
    html.Center(html.B(html.H2('Robert Lowrey'))), #Including name as my unique identifier 
    html.Hr(),
    html.Div(
        
# Code for the interactive filtering options
    className = 'Row',
            style = {'display': 'flex'},
            children ={
                html.Button(id = 'button1', n_clicks = 0, children = 'Water Rescue'), #Water Rescue filtering button
                html.Button(id = 'button2', n_clicks = 0, children = 'Mountain or Wilderness Rescue'), #Mountain or Wilderness Rescue button
                html.Button(id = 'button3', n_clicks = 0, children = 'Disaster or Individual Tracking'), #Disaster or Individual Tracking button
                html.Button(id = 'button4', n_clicks = 0, children = 'Reset') #Reset button
            }

    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # Setting up the features for the interactive data table to make it user-friendly for the client
        #completed code from Module Six milestone
        page_size = 15,
        row_selectable = "single",
        
                         

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback([Output('datatable-id','data')], #setting callback for button option being pressed
              [Input('button1', 'n_clicks'), Input ('button2', 'n-clicks'), Input ('button3', 'n_clicks'), Input('button4', 'n_clicks')])
def update_dashboard(button1, button2, button3, button4):
    if (int(button1) >= 1): #if button1 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Labrador Retriever Mix'},{'breed' : 'Chesapeake Bay Retriever'},{'breed' : 'Newfoundland'}]}, #will search these breeds
            {'sex_upon_outcome': 'Intact Female'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Female with training age between 26-156 weeks
        button2, button3, button4 = 0 #setting the other button options to 0
        
    elif (int(button2) >= 1): #if button2 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'German Shepherd'}, {'breed' : 'Alaskan Malamute'}, {'breed' : 'Old English Sheepdog'}, {'breed' : 'Sierian Husky'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button3, button4 =0 #set other button options to 0
    
    elif (int(button3) >= 1): #if button3 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Doberman Pinscher'}, {'breed' : 'German Shepherd'}, {'breed' : 'Golden Retriever'}, {'breed' : 'Bloodhound'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 20, 'gte': 300}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button2, button4 = 0 #set other button options to 0
    
    elif (int(button4)>= 1): #if button4 is chosen, 
        df = pd.DataFrame.from_records(db.read()), #reset the read method
        button1, button2, button3 = 0 #set other button options to zero
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns] #can select columns but not delete them
    data=df.to_dict('records') #setting data to dictionary

    return (data,columns) #return data and columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [ #adding code for pie chart that responds to filtering options
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:17406/


In [2]:
pip list

Package                           Version
--------------------------------- --------------------
aiohttp                           3.8.1
aiosignal                         1.2.0
alabaster                         0.7.12
anaconda-client                   1.9.0
anaconda-navigator                2.1.4
anaconda-project                  0.10.2
ansi2html                         1.5.2
anyio                             3.5.0
appdirs                           1.4.4
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.2
astroid                           2.6.6
astropy                           5.0.4
asttokens                         2.0.5
async-timeout                     4.0.1
atomicwrites                      1.4.0
attrs                             21.4.0
Automat                           20.2.0
autopep8                          1.6.0
Babel                             2.9.1
backcall                          0.2.0
backports.functoo

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip uninstall -y pymongo
pip uninstall -y jupyter_dash
pip uninstall -y dash_leaflet
pip uninstall -y dash
pip uninstall -y plotly
pip uninstall -y dash-table
pip uninstall -y base64
pip uninstall -y numpy
pip uninstall -y pandas
pip uninstall -y matplotlib

SyntaxError: invalid syntax (1652354352.py, line 1)

In [5]:
pip uninstall -y pymongo jupyter_dash dash_leaflet dash plotly dash-table base64 numpy pandas matplotlib

Found existing installation: pymongo 3.12.0
Uninstalling pymongo-3.12.0:
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.9/shutil.py", line 823, in move
    os.rename(src, real_dst)
PermissionError: [Errno 13] Permission denied: '/usr/local/anaconda/lib/python3.9/site-packages/bson' -> '/tmp/pip-uninstall-i4itglzb'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 173, in _main
    status = self.run(options, args)
  File "/usr/local/anaconda/lib/python3.9/site-packages/pip/_internal/commands/uninstall.py", line 93, in run
    uninstall_pathset = req.uninstall(
  File "/usr/local/anaconda/lib/python3.9/site-packages/pip/_internal/req/req_install.py", line 633, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/anaconda/lib/python3.9/site-packages/pip/_interna

In [6]:
pip uninstall -y dash-table

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install dash-table

Defaulting to user installation because normal site-packages is not writeable
  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash 2.8.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.8.1 requires dash-html-components==2.0.0, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pymongo jupyter_dash dash_leaflet dash plotly base64 numpy pandas matplotlib

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement base64 (from versions: none)
ERROR: No matching distribution found for base64
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps



# animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# My username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Robert Lowrey')

#Adding in Grazioso Salvare’s logo
image_filename = 'ProjectLogo.png' # replace with my image file name
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src ='data:image/png;base64,{}'.format(encoded_image.decode())),#placing HTML image tag
    html.Center(html.B(html.H2('Robert Lowrey'))), #Including name as my unique identifier 
    html.Hr(),
    html.Div(
        
# Code for the interactive filtering options
    className = 'Row',
            style = {'display': 'flex'},
            children ={
                html.Button(id = 'button1', n_clicks = 0, children = 'Water Rescue'), #Water Rescue filtering button
                html.Button(id = 'button2', n_clicks = 0, children = 'Mountain or Wilderness Rescue'), #Mountain or Wilderness Rescue button
                html.Button(id = 'button3', n_clicks = 0, children = 'Disaster or Individual Tracking'), #Disaster or Individual Tracking button
                html.Button(id = 'button4', n_clicks = 0, children = 'Reset') #Reset button
            }

    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # Setting up the features for the interactive data table to make it user-friendly for the client
        #completed code from Module Six milestone
        page_size = 15,
        row_selectable = "single",
        
                         

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback([Output('datatable-id','data')], #setting callback for button option being pressed
              [Input('button1', 'n_clicks'), Input ('button2', 'n-clicks'), Input ('button3', 'n_clicks'), Input('button4', 'n_clicks')])
def update_dashboard(button1, button2, button3, button4):
    if (int(button1) >= 1): #if button1 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Labrador Retriever Mix'},{'breed' : 'Chesapeake Bay Retriever'},{'breed' : 'Newfoundland'}]}, #will search these breeds
            {'sex_upon_outcome': 'Intact Female'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Female with training age between 26-156 weeks
        button2, button3, button4 = 0 #setting the other button options to 0
        
    elif (int(button2) >= 1): #if button2 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'German Shepherd'}, {'breed' : 'Alaskan Malamute'}, {'breed' : 'Old English Sheepdog'}, {'breed' : 'Sierian Husky'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button3, button4 =0 #set other button options to 0
    
    elif (int(button3) >= 1): #if button3 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Doberman Pinscher'}, {'breed' : 'German Shepherd'}, {'breed' : 'Golden Retriever'}, {'breed' : 'Bloodhound'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 20, 'gte': 300}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button2, button4 = 0 #set other button options to 0
    
    elif (int(button4)>= 1): #if button4 is chosen, 
        df = pd.DataFrame.from_records(db.read()), #reset the read method
        button1, button2, button3 = 0 #set other button options to zero
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns] #can select columns but not delete them
    data=df.to_dict('records') #setting data to dictionary

    return (data,columns) #return data and columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [ #adding code for pie chart that responds to filtering options
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


ServerSelectionTimeoutError: nv-desktop-services.apporto.com:30182: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 65750c92a6a277b9ec73facd, topology_type: Single, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 30182) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:30182: [Errno 111] Connection refused')>]>

In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps



# animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# My username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Robert Lowrey')

#Adding in Grazioso Salvare’s logo
image_filename = 'ProjectLogo.png' # replace with my image file name
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([children =[
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src ='data:image/png;base64,{}'.format(encoded_image.decode())),#placing HTML image tag
    html.Center(html.B(html.H2('Robert Lowrey'))), #Including name as my unique identifier 
    html.Hr(),
    html.Div(
        
# Code for the interactive filtering options
    className = 'Row',
            style = {'display': 'flex'},
            children ={
                html.Button(id = 'button1', n_clicks = 0, children = 'Water Rescue'), #Water Rescue filtering button
                html.Button(id = 'button2', n_clicks = 0, children = 'Mountain or Wilderness Rescue'), #Mountain or Wilderness Rescue button
                html.Button(id = 'button3', n_clicks = 0, children = 'Disaster or Individual Tracking'), #Disaster or Individual Tracking button
                html.Button(id = 'button4', n_clicks = 0, children = 'Reset') #Reset button
            }

    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # Setting up the features for the interactive data table to make it user-friendly for the client
        #completed code from Module Six milestone
        page_size = 15,
        row_selectable = "single",
        
                         

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
]

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback([Output('datatable-id','data')], #setting callback for button option being pressed
              [Input('button1', 'n_clicks'), Input ('button2', 'n-clicks'), Input ('button3', 'n_clicks'), Input('button4', 'n_clicks')])
def update_dashboard(button1, button2, button3, button4):
    if (int(button1) >= 1): #if button1 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Labrador Retriever Mix'},{'breed' : 'Chesapeake Bay Retriever'},{'breed' : 'Newfoundland'}]}, #will search these breeds
            {'sex_upon_outcome': 'Intact Female'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Female with training age between 26-156 weeks
        button2, button3, button4 = 0 #setting the other button options to 0
        
    elif (int(button2) >= 1): #if button2 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'German Shepherd'}, {'breed' : 'Alaskan Malamute'}, {'breed' : 'Old English Sheepdog'}, {'breed' : 'Sierian Husky'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button3, button4 =0 #set other button options to 0
    
    elif (int(button3) >= 1): #if button3 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Doberman Pinscher'}, {'breed' : 'German Shepherd'}, {'breed' : 'Golden Retriever'}, {'breed' : 'Bloodhound'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 20, 'gte': 300}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button2, button4 = 0 #set other button options to 0
    
    elif (int(button4)>= 1): #if button4 is chosen, 
        df = pd.DataFrame.from_records(db.read()), #reset the read method
        button1, button2, button3 = 0 #set other button options to zero
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns] #can select columns but not delete them
    data=df.to_dict('records') #setting data to dictionary

    return (data,columns) #return data and columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [ #adding code for pie chart that responds to filtering options
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=False)


In [2]:
pip list

Package                           Version
--------------------------------- --------------------
aiohttp                           3.8.1
aiosignal                         1.2.0
alabaster                         0.7.12
anaconda-client                   1.9.0
anaconda-navigator                2.1.4
anaconda-project                  0.10.2
ansi2html                         1.5.2
anyio                             3.5.0
appdirs                           1.4.4
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.2
astroid                           2.6.6
astropy                           5.0.4
asttokens                         2.0.5
async-timeout                     4.0.1
atomicwrites                      1.4.0
attrs                             21.4.0
Automat                           20.2.0
autopep8                          1.6.0
Babel                             2.9.1
backcall                          0.2.0
backports.functoo

PyJWT                             2.1.0
pylint                            2.9.6
pyls-spyder                       0.4.0
pymongo                           3.12.0
pyodbc                            4.0.32
pyOpenSSL                         21.0.0
pyparsing                         3.0.4
pyrsistent                        0.18.0
PySocks                           1.7.1
pytest                            7.1.1
python-dateutil                   2.8.2
python-lsp-black                  1.0.0
python-lsp-jsonrpc                1.0.0
python-lsp-server                 1.2.4
python-slugify                    5.0.2
python-snappy                     0.6.0
pytz                              2021.3
pyviz-comms                       2.0.2
PyWavelets                        1.3.0
pyxdg                             0.27
PyYAML                            6.0
pyzmq                             22.3.0
QDarkStyle                        3.0.2
qstylizer                         0.1.10
QtAwesome                         1.

In [3]:
pip install pymongo jupyter_dash dash_leaflet dash plotly dash-table base64 numpy pandas matplotlib

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement base64 (from versions: none)
ERROR: No matching distribution found for base64
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install base64 1.3.1

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement base64 (from versions: none)
ERROR: No matching distribution found for base64
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install base64 -v 1.3.1

Using pip 21.2.4 from /usr/local/anaconda/lib/python3.9/site-packages/pip (python 3.9)
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement base64 (from versions: none)
ERROR: No matching distribution found for base64
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pybase64

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 64 kB 1.2 MB/s eta 0:00:011
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps



# animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# My username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Robert Lowrey')

#Adding in Grazioso Salvare’s logo
image_filename = 'ProjectLogo.png' # replace with my image file name
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src ='data:image/png;base64,{}'.format(encoded_image.decode())),#placing HTML image tag
    html.Center(html.B(html.H2('Robert Lowrey'))), #Including name as my unique identifier 
    html.Hr(),
    html.Div(
        
# Code for the interactive filtering options
    className = 'Row',
            style = {'display': 'flex'},
            children ={
                html.Button(id = 'button1', n_clicks = 0, children = 'Water Rescue'), #Water Rescue filtering button
                html.Button(id = 'button2', n_clicks = 0, children = 'Mountain or Wilderness Rescue'), #Mountain or Wilderness Rescue button
                html.Button(id = 'button3', n_clicks = 0, children = 'Disaster or Individual Tracking'), #Disaster or Individual Tracking button
                html.Button(id = 'button4', n_clicks = 0, children = 'Reset') #Reset button
            }

    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # Setting up the features for the interactive data table to make it user-friendly for the client
        #completed code from Module Six milestone
        page_size = 15,
        row_selectable = "single",
        
                         

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback([Output('datatable-id','data')], #setting callback for button option being pressed
              [Input('button1', 'n_clicks'), Input ('button2', 'n-clicks'), Input ('button3', 'n_clicks'), Input('button4', 'n_clicks')])
def update_dashboard(button1, button2, button3, button4):
    if (int(button1) >= 1): #if button1 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Labrador Retriever Mix'},{'breed' : 'Chesapeake Bay Retriever'},{'breed' : 'Newfoundland'}]}, #will search these breeds
            {'sex_upon_outcome': 'Intact Female'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Female with training age between 26-156 weeks
        button2, button3, button4 = 0 #setting the other button options to 0
        
    elif (int(button2) >= 1): #if button2 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'German Shepherd'}, {'breed' : 'Alaskan Malamute'}, {'breed' : 'Old English Sheepdog'}, {'breed' : 'Sierian Husky'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button3, button4 =0 #set other button options to 0
    
    elif (int(button3) >= 1): #if button3 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Doberman Pinscher'}, {'breed' : 'German Shepherd'}, {'breed' : 'Golden Retriever'}, {'breed' : 'Bloodhound'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 20, 'gte': 300}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button2, button4 = 0 #set other button options to 0
    
    elif (int(button4)>= 1): #if button4 is chosen, 
        df = pd.DataFrame.from_records(db.read()), #reset the read method
        button1, button2, button3 = 0 #set other button options to zero
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns] #can select columns but not delete them
    data=df.to_dict('records') #setting data to dictionary

    return (data,columns) #return data and columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [ #adding code for pie chart that responds to filtering options
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    df = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


ServerSelectionTimeoutError: nv-desktop-services.apporto.com:30182: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 657511df582ce00fa2d4ba9d, topology_type: Single, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 30182) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:30182: [Errno 111] Connection refused')>]>

In [7]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps



# animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# My username and password and CRUD Python module name

username = "aacuser"
passwrd = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, passwrd)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Robert Lowrey')

#Adding in Grazioso Salvare’s logo
image_filename = 'ProjectLogo.png' # replace with my image file name
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src ='data:image/png;base64,{}'.format(encoded_image.decode())),#placing HTML image tag
    html.Center(html.B(html.H2('Robert Lowrey'))), #Including name as my unique identifier 
    html.Hr(),
    html.Div(
        
# Code for the interactive filtering options
    className = 'Row',
            style = {'display': 'flex'},
            children ={
                html.Button(id = 'button1', n_clicks = 0, children = 'Water Rescue'), #Water Rescue filtering button
                html.Button(id = 'button2', n_clicks = 0, children = 'Mountain or Wilderness Rescue'), #Mountain or Wilderness Rescue button
                html.Button(id = 'button3', n_clicks = 0, children = 'Disaster or Individual Tracking'), #Disaster or Individual Tracking button
                html.Button(id = 'button4', n_clicks = 0, children = 'Reset') #Reset button
            }

    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # Setting up the features for the interactive data table to make it user-friendly for the client
        #completed code from Module Six milestone
        page_size = 15,
        row_selectable = "single",
        
                         

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback([Output('datatable-id','data')], #setting callback for button option being pressed
              [Input('button1', 'n_clicks'), Input ('button2', 'n-clicks'), Input ('button3', 'n_clicks'), Input('button4', 'n_clicks')])
def update_dashboard(button1, button2, button3, button4):
    if (int(button1) >= 1): #if button1 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Labrador Retriever Mix'},{'breed' : 'Chesapeake Bay Retriever'},{'breed' : 'Newfoundland'}]}, #will search these breeds
            {'sex_upon_outcome': 'Intact Female'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Female with training age between 26-156 weeks
        button2, button3, button4 = 0 #setting the other button options to 0
        
    elif (int(button2) >= 1): #if button2 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'German Shepherd'}, {'breed' : 'Alaskan Malamute'}, {'breed' : 'Old English Sheepdog'}, {'breed' : 'Sierian Husky'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button3, button4 =0 #set other button options to 0
    
    elif (int(button3) >= 1): #if button3 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Doberman Pinscher'}, {'breed' : 'German Shepherd'}, {'breed' : 'Golden Retriever'}, {'breed' : 'Bloodhound'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 20, 'gte': 300}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button2, button4 = 0 #set other button options to 0
    
    elif (int(button4)>= 1): #if button4 is chosen, 
        df = pd.DataFrame.from_records(db.read()), #reset the read method
        button1, button2, button3 = 0 #set other button options to zero
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns] #can select columns but not delete them
    data=df.to_dict('records') #setting data to dictionary

    return (data,columns) #return data and columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [ #adding code for pie chart that responds to filtering options
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


ServerSelectionTimeoutError: nv-desktop-services.apporto.com:30182: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 65751466582ce00fa2d4baa1, topology_type: Single, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 30182) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:30182: [Errno 111] Connection refused')>]>

In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps



# animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# My username and password and CRUD Python module name

user = "aacuser"
passwd = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(user, passwd)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Robert Lowrey')

#Adding in Grazioso Salvare’s logo
image_filename = 'ProjectLogo.png' # replace with my image file name
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src ='data:image/png;base64,{}'.format(encoded_image.decode())),#placing HTML image tag
    html.Center(html.B(html.H2('Robert Lowrey'))), #Including name as my unique identifier 
    html.Hr(),
    html.Div(
        
# Code for the interactive filtering options
    className = 'Row',
            style = {'display': 'flex'},
            children ={
                html.Button(id = 'button1', n_clicks = 0, children = 'Water Rescue'), #Water Rescue filtering button
                html.Button(id = 'button2', n_clicks = 0, children = 'Mountain or Wilderness Rescue'), #Mountain or Wilderness Rescue button
                html.Button(id = 'button3', n_clicks = 0, children = 'Disaster or Individual Tracking'), #Disaster or Individual Tracking button
                html.Button(id = 'button4', n_clicks = 0, children = 'Reset') #Reset button
            }

    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # Setting up the features for the interactive data table to make it user-friendly for the client
        #completed code from Module Six milestone
        page_size = 15,
        row_selectable = "single",
        
                         

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback([Output('datatable-id','data')], #setting callback for button option being pressed
              [Input('button1', 'n_clicks'), Input ('button2', 'n-clicks'), Input ('button3', 'n_clicks'), Input('button4', 'n_clicks')])
def update_dashboard(button1, button2, button3, button4):
    if (int(button1) >= 1): #if button1 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Labrador Retriever Mix'},{'breed' : 'Chesapeake Bay Retriever'},{'breed' : 'Newfoundland'}]}, #will search these breeds
            {'sex_upon_outcome': 'Intact Female'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Female with training age between 26-156 weeks
        button2, button3, button4 = 0 #setting the other button options to 0
        
    elif (int(button2) >= 1): #if button2 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'German Shepherd'}, {'breed' : 'Alaskan Malamute'}, {'breed' : 'Old English Sheepdog'}, {'breed' : 'Sierian Husky'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button3, button4 =0 #set other button options to 0
    
    elif (int(button3) >= 1): #if button3 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Doberman Pinscher'}, {'breed' : 'German Shepherd'}, {'breed' : 'Golden Retriever'}, {'breed' : 'Bloodhound'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 20, 'gte': 300}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button2, button4 = 0 #set other button options to 0
    
    elif (int(button4)>= 1): #if button4 is chosen, 
        df = pd.DataFrame.from_records(db.read()), #reset the read method
        button1, button2, button3 = 0 #set other button options to zero
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns] #can select columns but not delete them
    data=df.to_dict('records') #setting data to dictionary

    return (data,columns) #return data and columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [ #adding code for pie chart that responds to filtering options
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps



# animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# My username and password and CRUD Python module name

user = "aacuser"
passwd = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(user, passwd)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Robert Lowrey')

#Adding in Grazioso Salvare’s logo
image_filename = 'ProjectLogo.png' # replace with my image file name
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div(children=[
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src ='data:image/png;base64,{}'.format(encoded_image.decode())),#placing HTML image tag
    html.Center(html.B(html.H2('Robert Lowrey'))), #Including name as my unique identifier 
    html.Hr(),
    html.Div(
        
# Code for the interactive filtering options
    className = 'Row',
            style = {'display': 'flex'},
            children ={
                html.Button(id = 'button1', n_clicks = 0, children = 'Water Rescue'), #Water Rescue filtering button
                html.Button(id = 'button2', n_clicks = 0, children = 'Mountain or Wilderness Rescue'), #Mountain or Wilderness Rescue button
                html.Button(id = 'button3', n_clicks = 0, children = 'Disaster or Individual Tracking'), #Disaster or Individual Tracking button
                html.Button(id = 'button4', n_clicks = 0, children = 'Reset') #Reset button
            }

    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # Setting up the features for the interactive data table to make it user-friendly for the client
        #completed code from Module Six milestone
        page_size = 15,
        row_selectable = "single",
        
                         

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback([Output('datatable-id','data')], #setting callback for button option being pressed
              [Input('button1', 'n_clicks'), Input ('button2', 'n-clicks'), Input ('button3', 'n_clicks'), Input('button4', 'n_clicks')])
def update_dashboard(button1, button2, button3, button4):
    if (int(button1) >= 1): #if button1 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Labrador Retriever Mix'},{'breed' : 'Chesapeake Bay Retriever'},{'breed' : 'Newfoundland'}]}, #will search these breeds
            {'sex_upon_outcome': 'Intact Female'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Female with training age between 26-156 weeks
        button2, button3, button4 = 0 #setting the other button options to 0
        
    elif (int(button2) >= 1): #if button2 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'German Shepherd'}, {'breed' : 'Alaskan Malamute'}, {'breed' : 'Old English Sheepdog'}, {'breed' : 'Sierian Husky'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 26, 'gte': 156}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button3, button4 =0 #set other button options to 0
    
    elif (int(button3) >= 1): #if button3 is pressed
        df = pd.DataFrame.from_records(db.read({'$and' :[ #Will search for these key breeds according to the specifications document
            {'$or': [{'breed' : 'Doberman Pinscher'}, {'breed' : 'German Shepherd'}, {'breed' : 'Golden Retriever'}, {'breed' : 'Bloodhound'}, {'breed' : 'Rottweiler'}]}, #will search these preferred breeds
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$lte': 20, 'gte': 300}}]})) #will search for Intact Male with training age between 26-156 weeks
        button1, button2, button4 = 0 #set other button options to 0
    
    elif (int(button4)>= 1): #if button4 is chosen, 
        df = pd.DataFrame.from_records(db.read()), #reset the read method
        button1, button2, button3 = 0 #set other button options to zero
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns] #can select columns but not delete them
    data=df.to_dict('records') #setting data to dictionary

    return (data,columns) #return data and columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [ #adding code for pie chart that responds to filtering options
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


ServerSelectionTimeoutError: nv-desktop-services.apporto.com:30182: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 65751957e1b6307df0565177, topology_type: Single, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 30182) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:30182: [Errno 111] Connection refused')>]>